<a href="https://colab.research.google.com/github/creativesar/Guess-the-number-Computer-/blob/main/Guess_the_number_by_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

def guess_the_number():
    # Generate a random number between 1 and 100
    secret_number = random.randint(1, 100)
    attempts = 0
    max_attempts = 10

    print("Welcome to the Guess the Number game!")
    print(f"I'm thinking of a number between 1 and 100. You have {max_attempts} attempts.")

    while attempts < max_attempts:
        # Get user's guess
        try:
            guess = int(input("Enter your guess: "))
        except ValueError:
            print("Please enter a valid number.")
            continue

        attempts += 1

        # Check the guess
        if guess < secret_number:
            print("Too low!")
        elif guess > secret_number:
            print("Too high!")
        else:
            print(f"Congratulations! You guessed the number in {attempts} attempts!")
            break

        # Inform player of remaining attempts
        remaining = max_attempts - attempts
        if remaining > 0:
            print(f"You have {remaining} attempts left.")

    if attempts >= max_attempts and guess != secret_number:
        print(f"Game over! The number was {secret_number}.")

    # Ask if player wants to play again
    play_again = input("Do you want to play again? (yes/no): ").lower()
    if play_again == "yes" or play_again == "y":
        guess_the_number()
    else:
        print("Thanks for playing! Goodbye!")

if __name__ == "__main__":
    guess_the_number()

Welcome to the Guess the Number game!
I'm thinking of a number between 1 and 100. You have 10 attempts.
